In [482]:
import os
import sys
import json
import urllib
import zipfile
from collections import defaultdict
from datetime import datetime
import tempfile

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from azureml.core import Workspace, Model, Dataset

from tqdm.notebook import tqdm

from IPython.display import display

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from P10_03_luis.utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [254]:
# On définit les variables globales
JSON_PATH = "data/json/"
MODELS_PATH = "data/models/"

os.makedirs(JSON_PATH, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)

RANDOM_SEED = 42

In [113]:
def pprint_dict(data: dict):
    """Pretty print a dict"""
    print(json.dumps(data, indent=2))

# Introduction

## Le jeu de données

In [9]:
# Lien vers le dataset
DATASET_URL = "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+10%C2%A0-+D%C3%A9veloppez+un+chatbot+pour+r%C3%A9server+des+vacances/frames.zip"
DATASET_FN = "frames.json"

# On vérifie si le fichier est bien présent
if DATASET_FN in os.listdir(JSON_PATH):
    print("Tous les fichiers sont bien présents.")
# Sinon on télécharge et on extrait le fichier
else:
    print("Téléchargement des données en cours...")

    # On télécharge le .zip dans un fichier temporaire et on extrait les données
    tmp, _ = urllib.request.urlretrieve(DATASET_URL)
    with zipfile.ZipFile(tmp, "r") as f:
        # On extrait le fichier json
        f.extract(DATASET_FN, JSON_PATH)

    # On supprime le fichier temporaire
    urllib.request.urlcleanup()
    
    print("Téléchargement des données terminé.")

Téléchargement des données en cours...
Téléchargement des données terminé.


# Chargement des ressources

## Chargement du workspace

In [2]:
# On charge l’espace de travail Azure Machine Learning existant
ws = Workspace.from_config()

## Chargement du magasin des données

In [116]:
# On charge le magasin de données par défaut
datastore = ws.get_default_datastore()

# Exploration et analyse des données

## Chargement du fichier

In [10]:
# On charge les frames
with open("data/json/frames.json") as f:
    frames = json.load(f)

In [13]:
frames[0]

{'user_id': 'U22HTHYNP',
 'turns': [{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
   'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}],
      'name': 'inform'},
     {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
       {'val': 'Caprica', 'key': 'or_city'},
       {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
       {'val': '8', 'key': 'n_adults'},
       {'val': '1700', 'key': 'budget'}],
      'name': 'inform'}],
    'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
      'name': 'inform'},
     {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
       {'val': 'Caprica', 'key': 'or_city'},
       {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
       {'val': '8', 'key': 'n_adults'},
       {'val': '1700', 'key': 'budget'}],
      'name': 'inform'}],
    'active_frame': 1,
    'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
 

In [15]:
list(frames[0].keys())

['user_id', 'turns', 'wizard_id', 'id', 'labels']

In [18]:
list(frames[0]["turns"][0].keys())

['text', 'labels', 'author', 'timestamp']

## Analyse des textes

In [330]:
# On affiche le premier dialogue
for turn in frames[0]["turns"]:
    print("-" * 80)
    print(turn.get("text", ""))

--------------------------------------------------------------------------------
I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.
--------------------------------------------------------------------------------
Hi...I checked a few options for you, and unfortunately, we do not currently have any trips that meet this criteria.  Would you like to book an alternate travel option?
--------------------------------------------------------------------------------
Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.
--------------------------------------------------------------------------------
I checked the availability for this date and there were no trips available.  Would you like to select some alternate dates?
--------------------------------------------------------------------------------
I have no flexibility for dates... but I can leave from Atl

In [12]:
# On affiche les premières utterances des 10 premiers dialogues
for i in range(10):
    print(frames[i].get("turns", [])[0].get("text", ""))

I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.
Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.
Hello there i am looking to go on a vacation with my family to Gotham City, can you help me?
Hi I'd like to go to Caprica from Busan, between Sunday August 21, 2016 and Wednesday August 31, 2016
Hello, I am looking to book a trip for 2 adults and 6 children for $21,300 or less. We are departing from Kochi for Denver.
Hey, i Want to go to St. Louis on the 17th of August
I'm looking for a trip to Gotham City leaving from Kakariko Village on Saturday, August 13, 2016. 3 adults for no more than $2400 USD.
Hello, I would like to book a 2-week trip leaving from Melbourne on August 27. I would like to go to Mannheim.
Hello, I am planning to book a trip to pittsborgh
Hi, I need to go to Mos Eisley for a wedding, leaving on Saturday, August 13, 2016 and returning on Tuesday, August 16, 2016. Pr

## Analyse des labels

In [34]:
pprint_dict(frames[0]["turns"][0]["labels"]["acts_without_refs"])

[
  {
    "args": [
      {
        "val": "book",
        "key": "intent"
      }
    ],
    "name": "inform"
  },
  {
    "args": [
      {
        "val": "Atlantis",
        "key": "dst_city"
      },
      {
        "val": "Caprica",
        "key": "or_city"
      },
      {
        "val": "Saturday, August 13, 2016",
        "key": "str_date"
      },
      {
        "val": "8",
        "key": "n_adults"
      },
      {
        "val": "1700",
        "key": "budget"
      }
    ],
    "name": "inform"
  }
]


In [25]:
keys = defaultdict(set)
for i in frames:
    for j in i["turns"][0]["labels"]["acts_without_refs"]:
        for k in j["args"]:
            try:
                keys[k["key"]].add(k.get("val", ""))
            except:
                print(k)

In [30]:
{k: len(v) for k, v in keys.items()}

{'intent': 1,
 'dst_city': 245,
 'or_city': 254,
 'str_date': 211,
 'n_adults': 139,
 'budget': 118,
 'end_date': 149,
 'n_children': 35,
 'max_duration': 42,
 'breakfast': 1,
 'flex': 2,
 'price': 1,
 'name': 1,
 'beach': 1,
 'seat': 3,
 'category': 14,
 'min_duration': 2,
 'park': 1,
 'count_dst_city': 1,
 'university': 1}

In [27]:
keys["intent"]

{'book'}

## Transformation des données

In [331]:
label_to_entity = {
    "or_city": "from_city",
    "dst_city": "to_city",
    "str_date": "from_dt",
    "end_date": "to_dt",
    "budget": "budget"
}

intent_name = "book_flight"

def turn_to_luis_utterance(turn, intent_name, label_to_entity):
    """"""
    text = turn["text"]

    entity_labels = []
    for i in turn["labels"]["acts_without_refs"]:
        for l in i["args"]:
            k = l["key"]
            v = l["val"]

            if k and v:
                if k in label_to_entity.keys():
                    start_char_index = text.lower().find(v.lower())
                    if start_char_index == -1:
                        continue
                    
                    end_char_index = start_char_index + len(v) - 1

                    entity_labels.append({
                        "startCharIndex": start_char_index,
                        "endCharIndex": end_char_index,
                        "entityName": label_to_entity[k],
                    })
    
    res = {
        "text": text,
        "intentName": intent_name,
        "entityLabels": entity_labels,
    }
    return res

res = turn_to_luis_utterance(frames[0]["turns"][0], intent_name, label_to_entity)
pprint_dict(res)

{
  "text": "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
  "intentName": "book_flight",
  "entityLabels": [
    {
      "startCharIndex": 27,
      "endCharIndex": 34,
      "entityName": "to_city"
    },
    {
      "startCharIndex": 41,
      "endCharIndex": 47,
      "entityName": "from_city"
    },
    {
      "startCharIndex": 52,
      "endCharIndex": 76,
      "entityName": "from_dt"
    },
    {
      "startCharIndex": 117,
      "endCharIndex": 120,
      "entityName": "budget"
    }
  ]
}


In [332]:
res = []
for frame in frames:
    user_turn_id = 0
    for turn in frame["turns"]:
        if turn["author"] == "user":
            row = {"user_turn_id": user_turn_id}
            user_turn_id += 1
            
            row.update(turn_to_luis_utterance(turn, intent_name, label_to_entity))
        
            res.append(row)

In [333]:
df = pd.DataFrame(res)
df["entity_total_nb"] = df["entityLabels"].apply(len)

for entity_name in label_to_entity.values():
    df[f"{entity_name}_nb"] = df["entityLabels"].apply(
        lambda x: len(list(
            filter(lambda x1: x1["entityName"] == entity_name, x)
        ))
    )

df["text_word_nb"] = df["text"].apply(lambda x: len(x.split()))

none_intent_mask = df["entity_total_nb"] == 0
df.loc[none_intent_mask, "intentName"] = "None"
    
df.shape

(10407, 11)

In [334]:
df.head()

,user_turn_id,text,intentName,entityLabels,entity_total_nb,from_city_nb,to_city_nb,from_dt_nb,to_dt_nb,budget_nb,text_word_nb
0,0,I'd like to book a trip to Atlantis from Capri...,book_flight,"[{'startCharIndex': 27, 'endCharIndex': 34, 'e...",4,1,1,1,0,1,25
1,1,"Yes, how about going to Neverland from Caprica...",book_flight,"[{'startCharIndex': 24, 'endCharIndex': 32, 'e...",4,1,1,1,0,1,23
2,2,I have no flexibility for dates... but I can l...,book_flight,"[{'startCharIndex': 56, 'endCharIndex': 63, 'e...",2,2,0,0,0,0,18
3,3,I suppose I'll speak with my husband to see if...,None,[],0,0,0,0,0,0,25
4,0,"Hello, I am looking to book a vacation from Go...",book_flight,"[{'startCharIndex': 59, 'endCharIndex': 68, 'e...",3,1,1,0,0,1,16


In [335]:
df[
    ["text_word_nb", "entity_total_nb"] + [f"{i}_nb" for i in label_to_entity.values()]
].describe().T

,count,mean,std,min,25%,50%,75%,max
text_word_nb,10407.0,10.367061,8.633925,1.0,4.0,8.0,14.0,80.0
entity_total_nb,10407.0,0.766311,0.965664,0.0,0.0,1.0,1.0,5.0
from_city_nb,10407.0,0.180263,0.386917,0.0,0.0,0.0,0.0,2.0
to_city_nb,10407.0,0.317863,0.477689,0.0,0.0,0.0,1.0,5.0
from_dt_nb,10407.0,0.109926,0.316478,0.0,0.0,0.0,0.0,3.0
to_dt_nb,10407.0,0.067839,0.252245,0.0,0.0,0.0,0.0,2.0
budget_nb,10407.0,0.090420,0.287466,0.0,0.0,0.0,0.0,2.0


In [336]:
book_flight_utterances = df[(df["user_turn_id"] == 0) & (df["entity_total_nb"] == len(label_to_entity) - 1)]
len(book_flight_utterances)

128

In [337]:
none_utterances = df[(df["user_turn_id"] == 0) & (df["entity_total_nb"] == 0)]
len(none_utterances)

244

In [338]:
none_utterances = none_utterances.sample(len(book_flight_utterances), random_state=RANDOM_SEED)
len(none_utterances)

128

In [339]:
utterances = pd.concat([book_flight_utterances, none_utterances])

## Split des données

In [340]:
utterances_train = utterances.sample(frac=0.7, random_state=RANDOM_SEED)

In [341]:
utterances_test_idx = utterances.index.difference(utterances_train.index)
utterances_test = utterances.loc[utterances_test_idx]

In [342]:
utterances_train.shape, utterances_test.shape

((179, 11), (77, 11))

In [343]:
utterances_train = utterances_train[["text", "intentName", "entityLabels"]].to_dict("records")

In [344]:
pprint_dict(utterances_train[0])

{
  "text": "I'm exhausted and I need to get away. I want to treat myself to a five day trip.",
  "intentName": "None",
  "entityLabels": []
}


In [345]:
utterances_test = utterances_test[["text", "intentName", "entityLabels"]].to_dict("records")

In [346]:
from typing import List, Dict

from pydantic import BaseModel

In [347]:
class EntityBaseModel(BaseModel):
    entity: str
    startPos: int
    endPos: int
    children: List["EntityBaseModel"] = []

    def __init__(self, **kwargs):
        kwargs["entity"] = kwargs["entityName"]
        kwargs["startPos"] = kwargs["startCharIndex"]
        kwargs["endPos"] = kwargs["endCharIndex"]
        super().__init__(**kwargs)

class TestUterranceBaseModel(BaseModel):
    text: str
    intent: str
    entities: List[EntityBaseModel]

    def __init__(self, **kwargs):
        kwargs["intent"] = kwargs["intentName"]
        kwargs["entities"] = kwargs["entityLabels"]
        super().__init__(**kwargs)
        
def conv_example_to_utterance_format(utterance):
    """"""
    return TestUterranceBaseModel(**utterance).dict()

In [348]:
utterances_test = {
    "LabeledTestSetUtterances": list(map(conv_example_to_utterance_format, utterances_test))
}

In [349]:
utterances_test["LabeledTestSetUtterances"][0]

{'text': "Hi I'd like to go to Caprica from Busan, between Sunday August 21, 2016 and Wednesday August 31, 2016",
 'intent': 'book_flight',
 'entities': [{'entity': 'to_city',
   'startPos': 21,
   'endPos': 27,
   'children': []},
  {'entity': 'from_city', 'startPos': 34, 'endPos': 38, 'children': []},
  {'entity': 'from_dt', 'startPos': 49, 'endPos': 70, 'children': []},
  {'entity': 'to_dt', 'startPos': 76, 'endPos': 100, 'children': []}]}

## Enregistrement des datasets

In [353]:
with tempfile.TemporaryDirectory() as tmp_dir_name:
    # On enregistre les données
    file_path = os.path.join(tmp_dir_name, "utterances_train.json")
    with open(file_path, "w") as f:
        json.dump(utterances_train, f)
        
    file_path = os.path.join(tmp_dir_name, "utterances_test.json")
    with open(file_path, "w") as f:
        json.dump(utterances_test, f)
    
    # On upload tous les fichiers dans le datastore
    ds = Dataset.File.upload_directory(
        tmp_dir_name,
        target=(datastore, "utterances_2/" + datetime.now().strftime("%Y_%m_%d")),
        overwrite=True,
        show_progress=True
    )

Validating arguments.
Arguments validated.
Uploading file to utterances_2/2021_12_15
Uploading an estimated of 2 files
Uploading /tmp/tmp8tbi4wj_/utterances_test.json
Uploaded /tmp/tmp8tbi4wj_/utterances_test.json, 1 files out of an estimated total of 2
Uploading /tmp/tmp8tbi4wj_/utterances_train.json
Uploaded /tmp/tmp8tbi4wj_/utterances_train.json, 2 files out of an estimated total of 2
Uploaded 2 files
Creating new dataset


In [355]:
ds = ds.register(
    workspace=ws,
    name="utterances",
    description="Train and test utterances",
    create_new_version=True
)

## Chargement des données

In [356]:
with tempfile.TemporaryDirectory() as tmp_dir_name:
    dataset = Dataset.get_by_name(ws, name='utterances')
    dataset.download(target_path=tmp_dir_name, overwrite=False)
    
    file_path = os.path.join(tmp_dir_name, "utterances_train.json")
    with open(file_path) as f:
        train_json = json.load(f)
        
    file_path = os.path.join(tmp_dir_name, "utterances_test.json")
    with open(file_path) as f:
        test_json = json.load(f)

In [357]:
train_json[0]

{'text': "I'm exhausted and I need to get away. I want to treat myself to a five day trip.",
 'intentName': 'None',
 'entityLabels': []}

In [358]:
test_json["LabeledTestSetUtterances"][0]

{'text': "Hi I'd like to go to Caprica from Busan, between Sunday August 21, 2016 and Wednesday August 31, 2016",
 'intent': 'book_flight',
 'entities': [{'entity': 'to_city',
   'startPos': 21,
   'endPos': 27,
   'children': []},
  {'entity': 'from_city', 'startPos': 34, 'endPos': 38, 'children': []},
  {'entity': 'from_dt', 'startPos': 49, 'endPos': 70, 'children': []},
  {'entity': 'to_dt', 'startPos': 76, 'endPos': 100, 'children': []}]}

# Création du modèle LUIS

In [199]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject, AzureAccountInfoObject, LuisApp
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials

import json, time, uuid

In [200]:
from dotenv import load_dotenv, set_key

load_dotenv("../P10_03_luis/.env")

True

In [359]:
LUIS_AUTH_KEY = os.getenv("LUIS_AUTH_KEY")
LUIS_AUTH_ENDPOINT = os.getenv("LUIS_AUTH_ENDPOINT")

LUIS_PRED_KEY = os.getenv("LUIS_PRED_KEY")
LUIS_PRED_ENDPOINT = os.getenv("LUIS_PRED_ENDPOINT")

In [360]:
# On s'authentifie
client = LUISAuthoringClient(LUIS_AUTH_ENDPOINT, CognitiveServicesCredentials(LUIS_AUTH_KEY))

In [361]:
# We use a UUID to avoid name collisions.
app_name = "p10-luis-app"
app_version = "0.1"
intent_name = "book_flight"

In [362]:
# define app basics
app_def = ApplicationCreateObject(name=app_name, initial_version_id=app_version, culture='en-us')

# create app
app_id = client.apps.add(app_def)

# get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))

Created LUIS app with ID c75d938e-310d-41a1-88b5-5aad57f4cc9e


In [363]:
set_key("../P10_03_luis/.env", "LUIS_APP_ID", app_id)

(True, 'LUIS_APP_ID', 'c75d938e-310d-41a1-88b5-5aad57f4cc9e')

In [364]:
client.model.add_intent(app_id, app_version, intent_name)

'94e02d3a-8214-4ba2-b8cc-92ea6496f0ee'

In [365]:
# Add Prebuilt entity
client.model.add_prebuilt(
    app_id,
    app_version,
    prebuilt_extractor_names=[
        "number",
        "datetimeV2",
        "geographyV2"
    ]
)

In [366]:
entity_to_id = {
    "from_city": "",
    "to_city": "",
    "from_dt": "",
    "to_dt": "",
    "budget": ""
}

In [367]:
for entity_name in entity_to_id.keys():
    # add entity to app
    entity_to_id[entity_name] = client.model.add_entity(
        app_id,
        app_version,
        name=entity_name
    )

entity_to_id

{'from_city': '430b9696-a1ca-4d65-a77d-c465d136298b',
 'to_city': '955de702-2835-46f9-890e-5b971b20f07e',
 'from_dt': '022599e0-7e84-4a19-aebb-aee9c8c46d8f',
 'to_dt': '733f354c-ed03-4fbb-bd2c-a394f6905163',
 'budget': '34090e7b-f387-4f33-b410-396003871b6c'}

In [368]:
# add phrase list to app
client.features.add_phrase_list(
    app_id,
    app_version,
    {
        "enabledForAllModels": False,
        "isExchangeable": True,
        "name": "from_phrase_list",
        "phrases": "from,start at,begin from,leave"
    }
)

# add phrase list to app
client.features.add_phrase_list(
    app_id,
    app_version,
    {
        "enabledForAllModels": False,
        "isExchangeable": True,
        "name": "to_phrase_list",
        "phrases": "to,arrive,land at,go,going,stay,heading"
    }
)

5220478

In [369]:
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["to_city"],
    {"model_name": "geographyV2", "is_required": True}
)

# add phrase list as feature to subentity model
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["to_city"],
    {"feature_name": "to_phrase_list", "model_name": None}
)

In [370]:
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["from_city"],
    {"model_name": "geographyV2", "is_required": True}
)

# add phrase list as feature to subentity model
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["from_city"],
    {"feature_name": "from_phrase_list", "model_name": None}
)

In [373]:
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["budget"],
    {"model_name": "number", "is_required": True}
)

In [484]:
train_json

[{'text': "I'm exhausted and I need to get away. I want to treat myself to a five day trip.",
  'intentName': 'None',
  'entityLabels': []},
 {'text': "Hi. I'm looking for an adventure from Thursday, August 18 to Saturday, September 3. I'm thinking of going to Fortaleza. Yes, with free breakfast of course. I'm leaving from Essen.",
  'intentName': 'book_flight',
  'entityLabels': [{'startCharIndex': 38,
    'endCharIndex': 56,
    'entityName': 'from_dt'},
   {'startCharIndex': 61, 'endCharIndex': 81, 'entityName': 'to_dt'},
   {'startCharIndex': 109, 'endCharIndex': 117, 'entityName': 'to_city'},
   {'startCharIndex': 173, 'endCharIndex': 177, 'entityName': 'from_city'}]},
 {'text': 'you can help me find a trip. from miami\nto tijuana\nbetween august thirtieth until september third\none adult',
  'intentName': 'book_flight',
  'entityLabels': [{'startCharIndex': 34,
    'endCharIndex': 38,
    'entityName': 'from_city'},
   {'startCharIndex': 43, 'endCharIndex': 49, 'entityName': 'to_

In [485]:
# for utterance in train_json:
#     try:
#         client.examples.add(app_id, app_version, utterance, { "enableNestedChildren": True })
#     except:
#         print(json.dumps(utterance, indent=2))
        
client.examples.batch(
    app_id,
    app_version,
    train_utterances,
    enable_nested_children=True
)

ErrorResponseException: Operation returned an invalid status code 'Bad Request'

In [375]:
train(app_version)

Waiting 10 seconds for training to complete...
trained


In [ ]:
deploy(app_version, is_staging=True)

In [478]:
pred = get_prediction(
    is_staging=True,
    utterance="I want to go from Paris to London the 25/12/2021 and stay one week. I have only 512€."
)

pprint_dict(pred)

In [378]:
res = evaluate(is_staging=True, utterances=test_utterances)
res

## Enregistrement des paramètres du modèle

In [270]:
params = get_params(app_version)

{
  "source": [
    "('workspaceblobstore', 'utterances_2/2021_12_14')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "b50dcf59-f0d2-4636-a171-1f077f37c88c",
    "name": "utterances",
    "version": 1,
    "description": "Train and test utterances",
    "workspace": "Workspace.create(name='p10-ml-ws', subscription_id='82f2cbcd-fb58-4d87-a682-9782dff74a97', resource_group='p10-ml-rg')"
  }
}

In [ ]:
params["utterances"] = []
params["versionId"] = 0.1

In [271]:
model_config = {
    "model": params,
    "dataset": {
        "name": ds.name,
        "version": ds.version,
    }
}

In [273]:
file_path = os.path.join("../P10_03_luis", "params.json")
with open(file_path, "w") as f:
    json.dump(model_config, f)

## Suppression du modèle

In [237]:
delete(app_version)